<a href="https://colab.research.google.com/github/longevity-genie/just-agents/blob/main/examples/notebooks/03_coding_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coding Just-Agents tutorial

In this tutorial we will develop an LLM agent that can code with just-agents library.

It is the third tutorial in a series of tutorials, other ones are:
1. Basic agents tutorial
2. Database agent tutorial
3. Coding agent tutorial  (THIS ONE)

Here we will write an agent that can write code and install dependencies. The default just-agents code agent uses safe non-root docker sandboxes to isolate environment from the host. However, such approach will not work with google collab, so we will write a coding agent from scratch

Install just-agents library and provide API keys for Groq and OpenAi

In [11]:
!pip install litellm==1.67.4.post1
!pip install just-agents-core==0.7.2
!pip install just-agents-examples==0.7.2

In [12]:
import os
import getpass
from dotenv import load_dotenv

def get_api_keys():
  """Gets API keys from the user."""
  load_dotenv(override=True)

  # Check if GROQ_API_KEY is already set in environment
  groq_api_key = os.environ.get("GROQ_API_KEY")
  openai_api_key = os.environ.get("OPENAI_API_KEY")

  # Only prompt for key if not already set
  if not groq_api_key:
    print("Please enter your API keys:")
    groq_api_key = getpass.getpass("GROQ_API_KEY: ")
    # Set environment variable
    os.environ["GROQ_API_KEY"] = groq_api_key
    print("GROQ_API key set successfully.")
  else:
    print("GROQ API key already set in environment.")


# Call the function to get API keys from the user
get_api_keys()

GROQ API key already set in environment.


### Run functions to run code

Writing custom functions to run code and install dependencies

In [13]:
# Define the function
import io
import contextlib

def run_python_code(code: str):
    """
    code: str # python code to run
    """
    output = io.StringIO()
    error = io.StringIO()

    try:
        with contextlib.redirect_stdout(output), contextlib.redirect_stderr(error):
            exec(code)
        result = output.getvalue()
        if result:
            return result.strip()
        else:
            return "No output."
    except Exception as e:
        return f"Error: {str(e)}"

Let's test that code generation works

In [14]:
run_python_code("print('Hello World')")

'Hello World'

In [15]:
from just_agents import llm_options
from just_agents.base_agent import BaseAgent

prompt = "Compute 2 * 10 + (342 +3) / 23.0"

agent: BaseAgent = BaseAgent(
    llm_options=llm_options.LLAMA3_3,
    tools=[run_python_code]
)
response = agent.query(prompt)
f"""MODEL RESPONSE = "{response}" """
agent.memory.pretty_print_all_messages()

╭─────────────────────────────────────────────────── [System] ────────────────────────────────────────────────────╮
│ You are a helpful AI assistant                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── [User] ─────────────────────────────────────────────────────╮
│ Compute 2 * 10 + (342 +3) / 23.0                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── [Assistant] ──────────────────────────────────────────────────╮
│ Tool Call to run_python_code:                                                                                   │
│ {"code": "print(2 * 10 + (342 + 3) / 23.0)"}                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── [Tool] ─────────────────────────────────────────────────────╮
│ Response from run_python_code:                                                                                  │
│ 35.0                                                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── [Assistant] ──────────────────────────────────────────────────╮
│ The result of the computation is 35.0.                                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

# Dependency hell management

But what if it does not have dependencies installed?

In [16]:
agent.query("translate ATGCATAGGTAG with biopython")

'The translation of the DNA sequence "ATGCATAGGTAG" is "MHR*".'

In [17]:
import subprocess

def execute_bash(command: str):
    """
    Executes a Bash command and returns the output or error message.

    Parameters:
    command (st): Bash command as a string.

    Returns:
    str: The output or error message from executing the command.
    """
    try:
        result = subprocess.run(command, shell=True, text=True, capture_output=True)
        if result.returncode == 0:
            return result.stdout.strip() if result.stdout else "No output."
        else:
            return f"Error: {result.stderr.strip()}"
    except Exception as e:
        return f"Exception: {str(e)}"

# Example usage:
bash_command = "ls -la"
output = execute_bash(bash_command)
print(output)


total 16
drwxr-xr-x 1 root root 4096 May 14 13:38 .
drwxr-xr-x 1 root root 4096 May 21 07:52 ..
drwxr-xr-x 4 root root 4096 May 14 13:38 .config
drwxr-xr-x 1 root root 4096 May 14 13:38 sample_data


In [18]:
from just_agents import llm_options
from just_agents.base_agent import BaseAgent
import pprint


agent: BaseAgent = BaseAgent(
    llm_options=llm_options.OPENAI_GPT4oMINI,
    tools=[run_python_code, execute_bash]
)
agent.memory.add_on_message(lambda m: pprint.pprint(m))

result = agent.query("Install biopython with pip")
result

{'content': 'Install biopython with pip', 'role': user}
{'content': 'You are a helpful AI assistant', 'role': system}
{'annotations': [],
 'role': 'assistant',
 'tool_calls': [{'function': {'arguments': '{"command":"pip install '
                                           'biopython"}',
                              'name': 'execute_bash'},
                 'id': 'call_D9ObQGAZsGQpUYntGtzdHYD7',
                 'type': 'function'}]}
{'content': 'Requirement already satisfied: biopython in '
            '/usr/local/lib/python3.11/dist-packages (1.85)\n'
            'Requirement already satisfied: numpy in '
            '/usr/local/lib/python3.11/dist-packages (from biopython) (2.0.2)',
 'name': 'execute_bash',
 'role': 'tool',
 'tool_call_id': 'call_D9ObQGAZsGQpUYntGtzdHYD7'}
{'annotations': [],
 'content': 'Biopython is already installed on your system. The installed '
            'version is 1.85, and it also requires NumPy, which is already '
            'satisfied with version 2.0.

'Biopython is already installed on your system. The installed version is 1.85, and it also requires NumPy, which is already satisfied with version 2.0.2.'

In [19]:
agent.query("translate ATGCATAGGTAG with biopython")
agent.memory.pretty_print_all_messages()

{'content': 'translate ATGCATAGGTAG with biopython', 'role': user}
{'content': 'You are a helpful AI assistant', 'role': system}
{'annotations': [],
 'role': 'assistant',
 'tool_calls': [{'function': {'arguments': '{"code":"from Bio.Seq import '
                                           'Seq\\n\\n# Create a sequence '
                                           'object\\nsequence = '
                                           "Seq('ATGCATAGGTAG')\\n\\n# "
                                           'Translate the '
                                           'sequence\\ntranslated_sequence = '
                                           'sequence.translate()\\ntranslated_sequence"}',
                              'name': 'run_python_code'},
                 'id': 'call_gSkJvYJSDVNulMszzw1MXdFs',
                 'type': 'function'}]}
{'content': 'No output.',
 'name': 'run_python_code',
 'role': 'tool',
 'tool_call_id': 'call_gSkJvYJSDVNulMszzw1MXdFs'}
{'annotations': [],
 'content': 'It 

╭─────────────────────────────────────────────────── [System] ────────────────────────────────────────────────────╮
│ You are a helpful AI assistant                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── [User] ─────────────────────────────────────────────────────╮
│ Install biopython with pip                                                                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── [Assistant] ──────────────────────────────────────────────────╮
│ Tool Call to execute_bash:                                                                                      │
│ {"command":"pip install biopython"}                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── [Tool] ─────────────────────────────────────────────────────╮
│ Response from execute_bash:                                                                                     │
│ Requirement already satisfied: biopython in /usr/local/lib/python3.11/dist-packages (1.85)                      │
│ Requirement already satisfied: numpy in /usr/local/lib/python3.11/dist-packages (from biopython) (2.0.2)        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── [Assistant] ──────────────────────────────────────────────────╮
│ Tool Call to run_python_code:                                                                                   │
│ {"code":"from Bio.Seq import Seq\n\n# Create a sequence object\nsequence = Seq('ATGCATAGGTAG')\n\n# Translate   │
│ the sequence\ntranslated_sequence = sequence.translate()\n\n# Output the translated                             │
│ sequence\nstr(translated_sequence)"}                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── [Tool] ─────────────────────────────────────────────────────╮
│ Response from run_python_code:                                                                                  │
│ No output.                                                                                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── [Assistant] ──────────────────────────────────────────────────╮
│ Tool Call to run_python_code:                                                                                   │
│ {"code":"from Bio.Seq import Seq\n\n# Create a sequence object\nsequence = Seq('ATGCATAGGTAG')\n\n# Translate   │
│ the sequence\ntranslated_sequence = sequence.translate()\n\n# Output the translated                             │
│ sequence\nprint(translated_sequence)"}                                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── [Tool] ─────────────────────────────────────────────────────╮
│ Response from run_python_code:                                                                                  │
│ MHR*                                                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── [Assistant] ──────────────────────────────────────────────────╮
│ The translation of the sequence "ATGCATAGGTAG" results in the peptide sequence "MHR*".                          │
│                                                                                                                 │
│ Here, "M" stands for Methionine, "H" for Histidine, "R" for Arginine, and "*" indicates a stop codon. This      │
│ means that the translation process encountered a stop codon, which terminates the translation.                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯